# Chapter 36: Testing TypeScript Code

Testing is essential for maintaining code quality and reliability in TypeScript applications. TypeScript's type system helps catch errors at compile time, but runtime behavior, logic correctness, and integration between components still require comprehensive testing. This chapter covers testing strategies, tools, and patterns specific to TypeScript across different environments.

---

## 36.1 Unit Testing with Jest and Vitest

Modern TypeScript projects primarily use Jest or Vitest for unit testing. Vitest has gained popularity for its native TypeScript support and Vite ecosystem compatibility.

### Jest Configuration for TypeScript

```typescript
// jest.config.ts
import type { Config } from 'jest';

const config: Config = {
  preset: 'ts-jest',
  testEnvironment: 'node', // or 'jsdom' for browser
  roots: ['<rootDir>/src'],
  testMatch: ['**/__tests__/**/*.test.ts', '**/?(*.)+(spec|test).ts'],
  transform: {
    '^.+\\.tsx?$': [
      'ts-jest',
      {
        tsconfig: '<rootDir>/tsconfig.spec.json',
        diagnostics: {
          ignoreCodes: [151001]
        }
      }
    ]
  },
  moduleNameMapper: {
    '^@/(.*)$': '<rootDir>/src/$1',
    '^@components/(.*)$': '<rootDir>/src/components/$1'
  },
  setupFilesAfterEnv: ['<rootDir>/src/test-setup.ts'],
  collectCoverageFrom: [
    'src/**/*.{ts,tsx}',
    '!src/**/*.d.ts',
    '!src/**/__tests__/**',
    '!src/**/index.ts'
  ],
  coverageThreshold: {
    global: {
      branches: 80,
      functions: 80,
      lines: 80,
      statements: 80
    }
  },
  clearMocks: true,
  restoreMocks: true
};

export default config;
```

**tsconfig.spec.json:**
```json
{
  "extends": "./tsconfig.json",
  "compilerOptions": {
    "target": "ES2020",
    "module": "commonjs",
    "jsx": "react-jsx",
    "types": ["jest", "node"]
  },
  "include": ["src/**/*", "src/**/*.test.ts"]
}
```

### Vitest Configuration (Recommended)

```typescript
// vitest.config.ts
import { defineConfig } from 'vitest/config';
import path from 'path';

export default defineConfig({
  test: {
    globals: true, // Enable global APIs (describe, it, expect)
    environment: 'jsdom', // or 'node'
    include: ['src/**/*.{test,spec}.{js,mjs,cjs,ts,mts,cts,jsx,tsx}'],
    exclude: ['**/node_modules/**', '**/dist/**'],
    setupFiles: ['./src/test/setup.ts'],
    coverage: {
      provider: 'v8',
      reporter: ['text', 'json', 'html'],
      reportsDirectory: './coverage',
      exclude: [
        'node_modules/',
        'src/test/',
        '**/*.d.ts',
        '**/*.config.*'
      ]
    },
    alias: {
      '@': path.resolve(__dirname, './src'),
      '@components': path.resolve(__dirname, './src/components')
    }
  },
  resolve: {
    alias: {
      '@': path.resolve(__dirname, './src')
    }
  }
});
```

### Writing Unit Tests

```typescript
// src/utils/calculator.ts
export class Calculator {
  add(a: number, b: number): number {
    return a + b;
  }

  subtract(a: number, b: number): number {
    return a - b;
  }

  divide(a: number, b: number): number {
    if (b === 0) {
      throw new Error('Division by zero');
    }
    return a / b;
  }

  async asyncAdd(a: number, b: number): Promise<number> {
    return new Promise((resolve) => {
      setTimeout(() => resolve(a + b), 100);
    });
  }
}

// src/utils/__tests__/calculator.test.ts
import { describe, it, expect, beforeEach } from 'vitest'; // or from '@jest/globals'
import { Calculator } from '../calculator';

describe('Calculator', () => {
  let calculator: Calculator;

  beforeEach(() => {
    calculator = new Calculator();
  });

  describe('add', () => {
    it('should add two positive numbers correctly', () => {
      // Arrange
      const a = 5;
      const b = 3;

      // Act
      const result = calculator.add(a, b);

      // Assert
      expect(result).toBe(8);
      expect(result).toBeTypeOf('number');
    });

    it('should handle negative numbers', () => {
      expect(calculator.add(-5, -3)).toBe(-8);
      expect(calculator.add(-5, 3)).toBe(-2);
    });
  });

  describe('divide', () => {
    it('should divide two numbers correctly', () => {
      expect(calculator.divide(10, 2)).toBe(5);
    });

    it('should throw error when dividing by zero', () => {
      // Type-safe error testing
      expect(() => calculator.divide(10, 0)).toThrow('Division by zero');
      expect(() => calculator.divide(10, 0)).toThrow(Error);
    });
  });

  describe('asyncAdd', () => {
    it('should resolve with correct sum', async () => {
      const result = await calculator.asyncAdd(5, 3);
      expect(result).toBe(8);
    });

    it('should handle async errors', async () => {
      // Testing async operations
      await expect(calculator.asyncAdd(5, 3)).resolves.toBe(8);
    });
  });
});
```

### Testing Pure Functions

```typescript
// src/utils/validators.ts
export interface ValidationResult<T> {
  isValid: boolean;
  errors: Partial<Record<keyof T, string>>;
  data?: T;
}

export function validateEmail(email: string): boolean {
  const emailRegex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  return emailRegex.test(email);
}

export function validateUserInput<T extends object>(
  data: unknown,
  schema: { [K in keyof T]: (value: unknown) => boolean }
): ValidationResult<T> {
  const errors: Partial<Record<keyof T, string>> = {};
  let isValid = true;

  if (!data || typeof data !== 'object') {
    return { isValid: false, errors: { root: 'Invalid data object' } as any };
  }

  const typedData = data as T;

  (Object.keys(schema) as Array<keyof T>).forEach((key) => {
    const validator = schema[key];
    const value = typedData[key];
    
    if (!validator(value)) {
      errors[key] = `Invalid value for ${String(key)}`;
      isValid = false;
    }
  });

  return {
    isValid,
    errors,
    data: isValid ? typedData : undefined
  };
}

// __tests__/validators.test.ts
import { describe, it, expect } from 'vitest';
import { validateEmail, validateUserInput } from '../validators';

interface TestUser {
  name: string;
  age: number;
  email: string;
}

describe('validateEmail', () => {
  it.each([
    ['test@example.com', true],
    ['user.name@domain.co.uk', true],
    ['invalid-email', false],
    ['@example.com', false],
    ['user@', false],
    ['', false]
  ])('validateEmail(%s) should return %s', (email, expected) => {
    expect(validateEmail(email)).toBe(expected);
  });
});

describe('validateUserInput', () => {
  const schema = {
    name: (v: unknown) => typeof v === 'string' && v.length > 0,
    age: (v: unknown) => typeof v === 'number' && v > 0 && v < 150,
    email: (v: unknown) => typeof v === 'string' && validateEmail(v)
  };

  it('should return valid for correct data', () => {
    const data = {
      name: 'John',
      age: 30,
      email: 'john@example.com'
    };

    const result = validateUserInput<TestUser>(data, schema);
    
    expect(result.isValid).toBe(true);
    expect(result.data).toEqual(data);
    expect(result.errors).toEqual({});
  });

  it('should return errors for invalid data', () => {
    const data = {
      name: '',
      age: -5,
      email: 'invalid'
    };

    const result = validateUserInput<TestUser>(data, schema);
    
    expect(result.isValid).toBe(false);
    expect(result.errors).toHaveProperty('name');
    expect(result.errors).toHaveProperty('age');
    expect(result.errors).toHaveProperty('email');
  });

  it('should handle non-object input', () => {
    const result = validateUserInput<TestUser>(null, schema);
    expect(result.isValid).toBe(false);
  });
});
```

---

## 36.2 TypeScript Testing Utilities

TypeScript requires specific utilities for type-safe mocking and assertions.

### Type-Safe Mocks

```typescript
// src/types/index.ts
export interface DatabaseService {
  connect(): Promise<void>;
  disconnect(): Promise<void>;
  query<T>(sql: string, params?: unknown[]): Promise<T[]>;
  transaction<T>(callback: (trx: DatabaseService) => Promise<T>): Promise<T>;
}

// src/__mocks__/database.ts
import { vi } from 'vitest';
import type { DatabaseService } from '../types';

export function createMockDatabaseService(): jest.Mocked<DatabaseService> {
  return {
    connect: vi.fn().mockResolvedValue(undefined),
    disconnect: vi.fn().mockResolvedValue(undefined),
    query: vi.fn().mockResolvedValue([]),
    transaction: vi.fn().mockImplementation(async (callback) => {
      return callback(createMockDatabaseService());
    })
  } as jest.Mocked<DatabaseService>;
}

// Alternative with Vitest
import { MockedObject } from 'vitest';

export function createVitestMock<T>(): MockedObject<T> {
  return {
    // Methods will be auto-mocked
  } as MockedObject<T>;
}
```

### Custom Matchers

```typescript
// src/test/matchers.ts
import { expect } from 'vitest';

interface CustomMatchers<R = unknown> {
  toBeValidUser(): R;
  toBeWithinRange(floor: number, ceiling: number): R;
  toBeTypeOf(expected: 'string' | 'number' | 'boolean' | 'object' | 'function'): R;
}

declare module 'vitest' {
  interface Assertion<T = any> extends CustomMatchers<T> {}
  interface AsymmetricMatchersContaining extends CustomMatchers {}
}

export function setupCustomMatchers() {
  expect.extend({
    toBeValidUser(received: unknown) {
      const { isNot } = this;
      const user = received as { name?: string; email?: string; age?: number };
      
      const hasName = typeof user?.name === 'string' && user.name.length > 0;
      const hasEmail = typeof user?.email === 'string' && user.email.includes('@');
      const hasValidAge = typeof user?.age === 'number' && user.age > 0;

      const pass = hasName && hasEmail && hasValidAge;

      return {
        pass,
        message: () =>
          isNot
            ? 'Expected user not to be valid'
            : `Expected user to be valid, but failed checks: ${
                [!hasName && 'name', !hasEmail && 'email', !hasValidAge && 'age']
                  .filter(Boolean)
                  .join(', ')
              }`
      };
    },

    toBeWithinRange(received: number, floor: number, ceiling: number) {
      const { isNot } = this;
      const pass = received >= floor && received <= ceiling;

      return {
        pass,
        message: () =>
          isNot
            ? `Expected ${received} not to be within range ${floor} - ${ceiling}`
            : `Expected ${received} to be within range ${floor} - ${ceiling}`
      };
    }
  });
}

// Usage in test
expect({ name: 'John', email: 'john@example.com', age: 30 }).toBeValidUser();
expect(5).toBeWithinRange(1, 10);
```

---

## 36.3 Mocking Strategies

Effective mocking is crucial for isolating units under test.

### Module Mocking

```typescript
// src/services/api.ts
export async function fetchUser(id: string): Promise<User> {
  const response = await fetch(`/api/users/${id}`);
  if (!response.ok) throw new Error('Failed to fetch');
  return response.json();
}

// __tests__/userService.test.ts
import { describe, it, expect, vi, beforeEach, afterEach } from 'vitest';
import * as apiModule from '../services/api';
import { getUserDetails } from '../services/userService';

// Mock the module
vi.mock('../services/api', () => ({
  fetchUser: vi.fn()
}));

describe('UserService with mocked API', () => {
  const mockFetchUser = vi.mocked(apiModule.fetchUser);

  beforeEach(() => {
    mockFetchUser.mockClear();
  });

  it('should return user details when API succeeds', async () => {
    const mockUser = { id: '1', name: 'John', email: 'john@example.com' };
    mockFetchUser.mockResolvedValue(mockUser);

    const result = await getUserDetails('1');
    
    expect(result).toEqual(mockUser);
    expect(mockFetchUser).toHaveBeenCalledWith('1');
    expect(mockFetchUser).toHaveBeenCalledTimes(1);
  });

  it('should throw error when API fails', async () => {
    mockFetchUser.mockRejectedValue(new Error('Network error'));

    await expect(getUserDetails('1')).rejects.toThrow('Network error');
  });

  it('should handle partial mocking', async () => {
    // Spy on original implementation
    const spy = vi.spyOn(apiModule, 'fetchUser').mockImplementation(async (id) => {
      if (id === '1') return { id: '1', name: 'Mocked' } as User;
      return apiModule.fetchUser(id); // Call original for other cases
    });
  });
});
```

### Class Mocking

```typescript
// src/services/emailService.ts
export class EmailService {
  private apiKey: string;
  
  constructor(apiKey: string) {
    this.apiKey = apiKey;
  }

  async sendEmail(to: string, subject: string, body: string): Promise<boolean> {
    // Implementation
    return true;
  }

  validateEmail(email: string): boolean {
    return email.includes('@');
  }
}

// __tests__/notificationService.test.ts
import { describe, it, expect, vi } from 'vitest';
import { EmailService } from '../services/emailService';
import { NotificationService } from '../services/notificationService';

// Manual mock class
class MockEmailService {
  sendEmail = vi.fn().mockResolvedValue(true);
  validateEmail = vi.fn().mockReturnValue(true);
}

describe('NotificationService', () => {
  it('should send notification using mocked email service', async () => {
    const mockEmailService = new MockEmailService() as unknown as EmailService;
    const notificationService = new NotificationService(mockEmailService);

    await notificationService.notifyUser('user@example.com', 'Hello');

    expect(mockEmailService.sendEmail).toHaveBeenCalledWith(
      'user@example.com',
      'Notification',
      'Hello'
    );
  });
});
```

---

## 36.4 Testing React Components

Testing React components with TypeScript requires rendering utilities and type-safe queries.

### React Testing Library Setup

```typescript
// src/test/test-utils.tsx
import React, { ReactElement } from 'react';
import { render, RenderOptions } from '@testing-library/react';
import { Provider } from 'react-redux'; // or your provider
import { store } from '../store';

// Custom render with providers
interface CustomRenderOptions extends Omit<RenderOptions, 'wrapper'> {
  initialState?: Partial<RootState>;
}

export function renderWithProviders(
  ui: ReactElement,
  { initialState, ...renderOptions }: CustomRenderOptions = {}
) {
  const Wrapper: React.FC<{ children: React.ReactNode }> = ({ children }) => {
    return <Provider store={store}>{children}</Provider>;
  };

  return {
    store,
    ...render(ui, { wrapper: Wrapper, ...renderOptions })
  };
}

// Re-export everything
export * from '@testing-library/react';
export { renderWithProviders as render };
```

### Component Testing

```typescript
// src/components/UserProfile.tsx
import React from 'react';

interface User {
  id: string;
  name: string;
  email: string;
  role: 'admin' | 'user';
}

interface UserProfileProps {
  user: User;
  onUpdate?: (user: Partial<User>) => void;
  loading?: boolean;
}

export const UserProfile: React.FC<UserProfileProps> = ({ 
  user, 
  onUpdate, 
  loading = false 
}) => {
  if (loading) return <div data-testid="loading">Loading...</div>;

  return (
    <div data-testid="user-profile">
      <h2>{user.name}</h2>
      <p data-testid="user-email">{user.email}</p>
      <span data-testid="user-role">{user.role}</span>
      {onUpdate && (
        <button 
          onClick={() => onUpdate({ name: 'Updated' })}
          data-testid="update-btn"
        >
          Update
        </button>
      )}
    </div>
  );
};

// __tests__/UserProfile.test.tsx
import React from 'react';
import { describe, it, expect, vi } from 'vitest';
import { screen, fireEvent } from '@testing-library/react';
import { UserProfile } from '../components/UserProfile';
import { render } from '../test/test-utils';

const mockUser = {
  id: '1',
  name: 'John Doe',
  email: 'john@example.com',
  role: 'admin' as const
};

describe('UserProfile', () => {
  it('renders user information correctly', () => {
    render(<UserProfile user={mockUser} />);
    
    expect(screen.getByTestId('user-profile')).toBeInTheDocument();
    expect(screen.getByText('John Doe')).toBeInTheDocument();
    expect(screen.getByTestId('user-email')).toHaveTextContent('john@example.com');
    expect(screen.getByTestId('user-role')).toHaveTextContent('admin');
  });

  it('shows loading state', () => {
    render(<UserProfile user={mockUser} loading={true} />);
    expect(screen.getByTestId('loading')).toBeInTheDocument();
  });

  it('handles update callback with correct types', () => {
    const handleUpdate = vi.fn();
    render(<UserProfile user={mockUser} onUpdate={handleUpdate} />);
    
    fireEvent.click(screen.getByTestId('update-btn'));
    
    expect(handleUpdate).toHaveBeenCalledWith(
      expect.objectContaining({ name: 'Updated' })
    );
  });

  it('type checks prevent invalid props', () => {
    // @ts-expect-error - role must be 'admin' | 'user'
    const invalidUser = { ...mockUser, role: 'superadmin' };
    
    // This would fail TypeScript checking
    // render(<UserProfile user={invalidUser} />);
  });
});
```

### Hook Testing

```typescript
// src/hooks/useCounter.ts
import { useState, useCallback } from 'react';

interface UseCounterReturn {
  count: number;
  increment: () => void;
  decrement: () => void;
  reset: (value?: number) => void;
}

export function useCounter(initialValue: number = 0): UseCounterReturn {
  const [count, setCount] = useState(initialValue);

  const increment = useCallback(() => setCount(c => c + 1), []);
  const decrement = useCallback(() => setCount(c => c - 1), []);
  const reset = useCallback((value: number = initialValue) => setCount(value), [initialValue]);

  return { count, increment, decrement, reset };
}

// __tests__/useCounter.test.ts
import { describe, it, expect } from 'vitest';
import { renderHook, act } from '@testing-library/react';
import { useCounter } from '../hooks/useCounter';

describe('useCounter', () => {
  it('should initialize with default value', () => {
    const { result } = renderHook(() => useCounter());
    expect(result.current.count).toBe(0);
  });

  it('should initialize with provided value', () => {
    const { result } = renderHook(() => useCounter(10));
    expect(result.current.count).toBe(10);
  });

  it('should increment', () => {
    const { result } = renderHook(() => useCounter());

    act(() => {
      result.current.increment();
    });

    expect(result.current.count).toBe(1);
  });

  it('should reset to initial value', () => {
    const { result } = renderHook(() => useCounter(5));

    act(() => {
      result.current.increment();
      result.current.reset();
    });

    expect(result.current.count).toBe(5);
  });

  it('should reset to specific value', () => {
    const { result } = renderHook(() => useCounter());

    act(() => {
      result.current.reset(100);
    });

    expect(result.current.count).toBe(100);
  });
});
```

---

## 36.5 Testing Vue Components

Vue Test Utils provides the testing infrastructure for Vue 3 with TypeScript.

### Vue Component Testing Setup

```typescript
// src/components/Counter.vue
<script setup lang="ts">
import { ref, computed } from 'vue';

const props = defineProps<{
  initial?: number;
  min?: number;
  max?: number;
}>();

const emit = defineEmits<{
  (e: 'update', value: number): void;
  (e: 'limit-reached', type: 'min' | 'max'): void;
}>();

const count = ref(props.initial ?? 0);

const isAtMin = computed(() => props.min !== undefined && count.value <= props.min);
const isAtMax = computed(() => props.max !== undefined && count.value >= props.max);

const increment = () => {
  if (isAtMax.value) {
    emit('limit-reached', 'max');
    return;
  }
  count.value++;
  emit('update', count.value);
};

const decrement = () => {
  if (isAtMin.value) {
    emit('limit-reached', 'min');
    return;
  }
  count.value--;
  emit('update', count.value);
};
</script>

<template>
  <div class="counter">
    <button 
      @click="decrement" 
      :disabled="isAtMin"
      data-testid="decrement"
    >
      -
    </button>
    <span data-testid="count">{{ count }}</span>
    <button 
      @click="increment" 
      :disabled="isAtMax"
      data-testid="increment"
    >
      +
    </button>
  </div>
</template>
```

```typescript
// src/components/__tests__/Counter.spec.ts
import { describe, it, expect, vi } from 'vitest';
import { mount, VueWrapper } from '@vue/test-utils';
import Counter from '../Counter.vue';

describe('Counter', () => {
  it('renders with default initial value', () => {
    const wrapper = mount(Counter);
    expect(wrapper.find('[data-testid="count"]').text()).toBe('0');
  });

  it('renders with provided initial value', () => {
    const wrapper = mount(Counter, {
      props: {
        initial: 10
      }
    });
    expect(wrapper.find('[data-testid="count"]').text()).toBe('10');
  });

  it('increments when button clicked', async () => {
    const wrapper = mount(Counter);
    
    await wrapper.find('[data-testid="increment"]').trigger('click');
    
    expect(wrapper.find('[data-testid="count"]').text()).toBe('1');
  });

  it('emits update event with correct type', async () => {
    const wrapper = mount(Counter);
    
    await wrapper.find('[data-testid="increment"]').trigger('click');
    
    expect(wrapper.emitted('update')).toBeTruthy();
    expect(wrapper.emitted('update')![0]).toEqual([1]);
  });

  it('respects max limit', async () => {
    const wrapper = mount(Counter, {
      props: {
        initial: 5,
        max: 5
      }
    });

    await wrapper.find('[data-testid="increment"]').trigger('click');
    
    expect(wrapper.find('[data-testid="count"]').text()).toBe('5');
    expect(wrapper.emitted('limit-reached')).toEqual([['max']]);
  });

  it('type checks props', () => {
    // @ts-expect-error - initial should be number
    mount(Counter, { props: { initial: 'string' } });
  });
});
```

### Testing Composables

```typescript
// src/composables/useLocalStorage.ts
import { ref, watch, type Ref } from 'vue';

export function useLocalStorage<T>(key: string, defaultValue: T): Ref<T> {
  const stored = localStorage.getItem(key);
  const data = ref<T>(stored ? JSON.parse(stored) : defaultValue) as Ref<T>;

  watch(data, (newValue) => {
    localStorage.setItem(key, JSON.stringify(newValue));
  }, { deep: true });

  return data;
}

// __tests__/useLocalStorage.spec.ts
import { describe, it, expect, beforeEach } from 'vitest';
import { useLocalStorage } from '../../composables/useLocalStorage';
import { ref, nextTick } from 'vue';

describe('useLocalStorage', () => {
  beforeEach(() => {
    localStorage.clear();
  });

  it('should return default value when no stored value exists', () => {
    const data = useLocalStorage('test-key', 'default');
    expect(data.value).toBe('default');
  });

  it('should return stored value when it exists', () => {
    localStorage.setItem('test-key', JSON.stringify('stored'));
    const data = useLocalStorage('test-key', 'default');
    expect(data.value).toBe('stored');
  });

  it('should update localStorage when value changes', async () => {
    const data = useLocalStorage('test-key', 'initial');
    data.value = 'updated';
    
    await nextTick();
    
    expect(localStorage.getItem('test-key')).toBe(JSON.stringify('updated'));
  });

  it('should work with complex objects', async () => {
    interface User {
      name: string;
      age: number;
    }
    
    const defaultUser: User = { name: 'John', age: 30 };
    const data = useLocalStorage<User>('user-key', defaultUser);
    
    data.value.age = 31;
    await nextTick();
    
    const stored = JSON.parse(localStorage.getItem('user-key')!);
    expect(stored.age).toBe(31);
  });
});
```

---

## 36.6 Testing Node.js APIs

Testing Express/Fastify endpoints requires HTTP assertion libraries.

### Supertest with TypeScript

```typescript
// src/app.ts
import express, { Application } from 'express';
import { userRouter } from './routes/users';

export function createApp(): Application {
  const app = express();
  
  app.use(express.json());
  app.use('/api/users', userRouter);
  
  return app;
}
```

```typescript
// src/routes/__tests__/users.test.ts
import request from 'supertest';
import { describe, it, expect, beforeAll, afterAll } from 'vitest';
import type { Application } from 'express';
import { createApp } from '../../app';
import { prisma } from '../../lib/prisma'; // Your DB client

describe('User Routes', () => {
  let app: Application;

  beforeAll(() => {
    app = createApp();
  });

  afterAll(async () => {
    await prisma.$disconnect();
  });

  describe('GET /api/users', () => {
    it('should return list of users', async () => {
      const response = await request(app)
        .get('/api/users')
        .expect('Content-Type', /json/)
        .expect(200);

      expect(Array.isArray(response.body)).toBe(true);
      expect(response.body[0]).toHaveProperty('id');
      expect(response.body[0]).toHaveProperty('email');
    });
  });

  describe('POST /api/users', () => {
    it('should create a new user', async () => {
      const newUser = {
        name: 'Test User',
        email: 'test@example.com',
        password: 'password123'
      };

      const response = await request(app)
        .post('/api/users')
        .send(newUser)
        .expect(201);

      expect(response.body).toMatchObject({
        name: newUser.name,
        email: newUser.email
      });
      expect(response.body).not.toHaveProperty('password'); // Security check
    });

    it('should validate required fields', async () => {
      const response = await request(app)
        .post('/api/users')
        .send({ name: 'Test' }) // Missing email and password
        .expect(400);

      expect(response.body).toHaveProperty('error');
    });
  });

  describe('Authentication', () => {
    it('should protect private routes', async () => {
      await request(app)
        .get('/api/users/profile')
        .expect(401);
    });

    it('should access private route with valid token', async () => {
      // Login first
      const loginRes = await request(app)
        .post('/api/auth/login')
        .send({ email: 'test@example.com', password: 'password123' });

      const token = loginRes.body.token;

      await request(app)
        .get('/api/users/profile')
        .set('Authorization', `Bearer ${token}`)
        .expect(200);
    });
  });
});
```

---

## 36.7 Integration Testing

Integration tests verify that multiple units work together correctly.

### Database Integration Tests

```typescript
// src/__tests__/integration/user-flow.test.ts
import { describe, it, expect, beforeAll, afterAll, beforeEach } from 'vitest';
import { createApp } from '../../app';
import { prisma } from '../../lib/prisma';
import type { Application } from 'express';
import request from 'supertest';

describe('User Registration Flow', () => {
  let app: Application;

  beforeAll(() => {
    app = createApp();
  });

  beforeEach(async () => {
    // Clean database before each test
    await prisma.user.deleteMany();
  });

  afterAll(async () => {
    await prisma.$disconnect();
  });

  it('should complete full registration and login flow', async () => {
    // 1. Register
    const registerRes = await request(app)
      .post('/api/auth/register')
      .send({
        name: 'Integration Test User',
        email: 'integration@test.com',
        password: 'securePassword123'
      })
      .expect(201);

    expect(registerRes.body).toHaveProperty('id');
    const userId = registerRes.body.id;

    // 2. Login
    const loginRes = await request(app)
      .post('/api/auth/login')
      .send({
        email: 'integration@test.com',
        password: 'securePassword123'
      })
      .expect(200);

    expect(loginRes.body).toHaveProperty('token');
    const token = loginRes.body.token;

    // 3. Access protected resource
    const profileRes = await request(app)
      .get('/api/users/profile')
      .set('Authorization', `Bearer ${token}`)
      .expect(200);

    expect(profileRes.body.id).toBe(userId);

    // 4. Update profile
    await request(app)
      .patch('/api/users/profile')
      .set('Authorization', `Bearer ${token}`)
      .send({ name: 'Updated Name' })
      .expect(200);

    // 5. Verify update
    const updatedProfile = await request(app)
      .get('/api/users/profile')
      .set('Authorization', `Bearer ${token}`)
      .expect(200);

    expect(updatedProfile.body.name).toBe('Updated Name');
  });
});
```

---

## 36.8 Property-Based Testing

Property-based testing generates random inputs to verify invariants.

```typescript
import { describe, it, expect } from 'vitest';
import fc from 'fast-check';

describe('Property-Based Tests', () => {
  it('addition should be commutative', () => {
    fc.assert(
      fc.property(
        fc.integer(),
        fc.integer(),
        (a, b) => {
          expect(a + b).toBe(b + a);
        }
      )
    );
  });

  it('sorting should maintain all elements', () => {
    fc.assert(
      fc.property(
        fc.array(fc.integer()),
        (arr) => {
          const sorted = [...arr].sort((a, b) => a - b);
          expect(sorted).toHaveLength(arr.length);
          expect(new Set(sorted)).toEqual(new Set(arr));
        }
      )
    );
  });

  it('email validator should reject invalid formats', () => {
    fc.assert(
      fc.property(
        fc.string(),
        (randomString) => {
          // Most random strings are not valid emails
          if (!randomString.includes('@') || !randomString.includes('.')) {
            expect(validateEmail(randomString)).toBe(false);
          }
        }
      )
    );
  });
});
```

---

## 36.9 Type Testing with tsd

`tsd` tests TypeScript type definitions to ensure types compile as expected.

```typescript
// src/types/index.test-d.ts
import { expectType, expectError } from 'tsd';
import { UserService } from './UserService';
import type { User, CreateUserDTO } from './types';

// Test that service methods return correct types
const service = {} as UserService;

expectType<Promise<User>>(service.getUserById('123'));
expectType<Promise<User[]>>(service.getAllUsers());

// Test that CreateUserDTO omits id
declare const dto: CreateUserDTO;
expectError(dto.id); // Should not have id property

// Test function parameter types
function processUser(user: User) {
  expectType<string>(user.id);
  expectType<string>(user.email);
  expectError(user.nonExistentProperty);
}
```

---

## 36.10 Chapter Summary and Exercises

### Chapter Summary

This chapter covered comprehensive testing strategies for TypeScript:

**Key Concepts:**

1. **Jest vs Vitest**: Vitest offers better TypeScript native support and Vite compatibility. Both use similar APIs but Vitest has faster HMR and native ESM support.

2. **Type Safety in Tests**: TypeScript catches type errors in test files, ensuring mocks match interfaces and assertions use correct types. `jest.mocked()` or `vi.mocked()` preserve types for mocked modules.

3. **Component Testing**: React Testing Library and Vue Test Utils provide type-safe utilities for rendering components, querying DOM elements, and simulating user interactions.

4. **Mocking Strategies**: Module mocking with `vi.mock()`, class mocking with manual mocks, and spy functions maintain type safety while isolating units under test.

5. **Integration Testing**: Supertest provides HTTP assertions for API testing. Database integration tests verify end-to-end flows with real (or test) databases.

6. **Property-Based Testing**: `fast-check` generates random test cases to verify invariants and edge cases that manual tests might miss.

7. **Type Testing**: `tsd` ensures type definitions work as intended, testing that certain code produces type errors while other code compiles successfully.

### Practical Exercises

**Exercise 1: Calculator with 100% Coverage**

Build a Calculator class with methods: add, subtract, multiply, divide, power, sqrt. Write tests achieving 100% code coverage including edge cases (division by zero, negative sqrt).

**Exercise 2: Type-Safe Mock Service**

Create a UserRepository interface and implement an InMemoryUserRepository for testing. Write tests for a UserService that depends on the repository, using the in-memory implementation for fast, isolated tests.

**Exercise 3: React Component Testing**

Build a SearchableList component that fetches data from an API and displays results with filtering. Write tests for:
- Loading states
- Empty results
- Error handling
- User interactions (search, sort, pagination)
- Accessibility attributes

**Exercise 4: Vue Composable Testing**

Create a `useFetch` composable that handles loading, error, and data states. Write comprehensive tests verifying:
- Initial state
- Successful data fetching
- Error handling
- Retry functionality
- Cancellation on unmount

**Exercise 5: API Integration Testing**

Set up an integration test suite for an Express API using a test database (SQLite or test container). Test complete user workflows including registration, authentication, CRUD operations, and authorization failures.

**Exercise 6: Property-Based Validation**

Write property-based tests for a password validator ensuring:
- Passwords with length < 8 are rejected
- Passwords without uppercase are rejected
- Passwords without numbers are rejected
- All valid combinations are accepted

### Additional Resources

- **Testing Library**: https://testing-library.com/docs/
- **Vitest Documentation**: https://vitest.dev/
- **Jest TypeScript**: https://jestjs.io/docs/getting-started#using-typescript
- **Supertest**: https://github.com/visionmedia/supertest
- **fast-check**: https://github.com/dubzzz/fast-check

---

## Coming Up Next: Chapter 37 - Linting and Formatting

In the next chapter, we will explore code quality tools for TypeScript:

- ESLint configuration for TypeScript (@typescript-eslint)
- Prettier integration and conflict resolution
- Strict linting rules for type safety
- Import/export organization
- Custom lint rules development
- Pre-commit hooks with Husky and lint-staged
- CI/CD integration for code quality
- TypeScript compiler checks vs linting

Proper linting and formatting ensures consistent code style across teams, catches potential bugs before runtime, and enforces TypeScript best practices automatically.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../9. typescript_with_frameworks/35. typescript_with_angular.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='37. linting_and_formatting.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
